#Predict score field by text and parent_text fields.

Вы не можете использовать тестовую выборку для обучения модели.
Результатом выполненного тестового задания является jupyter notebook c проведенным экспериментом. Клетки ноутбука должны быть выполнены по порядку сверху вниз. Должна быть обеспечена воспроизводимость эксперимента.

In [0]:
# https://towardsdatascience.com/predicting-amazon-reviews-scores-using-hierarchical-attention-networks-with-pytorch-and-apache-5214edb3df20
# https://arxiv.org/pdf/1901.07829.pdf

In [0]:
# TODO Пути вернуть в норм состояние
# TODO Разобраться с loss'ом 
# TODO Токенизация сейчас max=512. Как-то это разрезать надо бы.

In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import torch

SEED = 43
np.random.seed(SEED)

import warnings
warnings.filterwarnings('ignore')

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# Data Stuff

In [0]:
columns = ["text", "parent_text", "score"]
df = pd.concat([
    pd.read_csv("/content/drive/My Drive/jetbrains/comments_positive.csv", usecols=columns, na_filter=False),
    pd.read_csv("/content/drive/My Drive/jetbrains/comments_negative.csv", usecols=columns, na_filter=False)
], ignore_index=True)

In [0]:
y = df['score']
df.drop(columns='score', inplace=True)
X = df

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=SEED)

# To be sure we don't use indices to predict something
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

print("Train shape: {}".format(X_train.shape))
print("Test shape: {}".format(X_test.shape))

Train shape: (3800000, 2)
Test shape: (200000, 2)


In [6]:
X_train.head()

,text,parent_text
0,"I was a whole bunch of folks on Recess, yup. :...",Your IMDB just blew my mind. You were Upside D...
1,"I'm not quite sure how that was relevant, but ...",http://i.imgur.com/GKLI7.jpg
2,Sounds like you weren't paying attention.,Sadly high school history doesn't teach you ab...
3,&gt;Polygamy doesn't mean that there's just wo...,&gt; from a feminist perspective \n\nSo what i...
4,The New Orleans Gargle-Glub-Glubs.,My high school's mascot was water...


In [7]:
y_train.head()

0    454
1     71
2     -9
3    -12
4     91
Name: score, dtype: int64

In [9]:
# print(X_train['text'].str.len().mean(), X_train['parent_text'].str.len().mean())
# print(X_test['text'].str.len().mean(), X_test['parent_text'].str.len().mean())

171.01774210526315 218.5676302631579
170.53281 220.14226


In [0]:
# import nltk
# import spacy 

# nltk.download('punkt')
# spacy_en = spacy.load('en')


# X_t_l = X['text'].apply(lambda x: len(spacy_en.tokenizer(x)))
# X_t_p = X['parent_text'].apply(lambda x: len(spacy_en.tokenizer(x)))

# import numpy as np

# np.mean(np.array(X_t_l)), np.mean(np.array(X_t_p))

**В среднем - 37.4 и 47. токенов.**

## Cleaning the data

Т.к. некоторые тексты - это просто смайлики или ссылки на что-то, то их было решено удалить. Также были удалены строки, если после очисти они были пустыми. Всего из тренировочных данных было удалено около 600 строк. Из тестовых - 200. 

In [0]:
import re
import string

In [0]:
def clean_text(text):
    text = re.sub(r'\<[^>]*\>', '', text)
    text = re.sub(r'\\n', ' ', text)
    text = re.sub(r'&[a-z]{0,7};', ' ', text)
    text = re.sub(r'\s{2,10}', ' ', text)
    text = re.sub(r'\\x\d{1,4}', '', text)
    text = re.sub(r'\\n', ' ', text)
    text = re.sub(r'\r', ' ', text)
    text = re.sub(r'\s{2,10}', ' ', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

def clean_df(df, y, text_cleaner):
    """
    Applying text_cleaner to df columns and deleting rows with NaN
    Return pandas DataFrame with rating coumns as well
    """
    df_cleaned = pd.DataFrame()
    df_cleaned['text'] = df['text'].apply(lambda x: text_cleaner(x))
    df_cleaned['parent_text'] = df['parent_text'].apply(lambda x: text_cleaner(x))
    df_cleaned['rating'] = y    

    replacing = {'':np.nan, 'null':np.nan, 'NaN':np.nan, 'NA':np.nan}
    df_cleaned['text'].replace(to_replace=replacing, value=None, inplace=True)
    df_cleaned['parent_text'].replace(to_replace=replacing, value=None, inplace=True)
    df_cleaned.dropna(subset=['text', 'parent_text', 'rating'], inplace=True)
    return df_cleaned

In [0]:
train = clean_df(X_train, y_train, clean_text)
test = clean_df(X_test, y_test, clean_text)
train.shape, X_train.shape, test.shape, X_test.shape

In [0]:
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)

Убедимся, что ничего не съехало:

In [34]:
train.iloc[-1], X_train.iloc[-1], y_train.iloc[-1]

(text           4x the pixels has nothing to do with the resol...
 parent_text    The new GPU is 2x as powerful as the iPad 2s b...
 rating                                                        -8
 Name: 3799999, dtype: object,
 text           4x the pixels has nothing to do with the resol...
 parent_text    The new GPU is 2x as powerful as the iPad 2's,...
 Name: 3799999, dtype: object,
 -8)

In [35]:
test.iloc[-1], X_test.iloc[-1], y_test.iloc[-1]

(text           Ha people on Reddit go on the side of whoever ...
 parent_text    I just hope you guys dont ruin it for the play...
 rating                                                        91
 Name: 199999, dtype: object,
 text           Ha, people on Reddit go on the side of whoever...
 parent_text    I just hope you guys don't ruin it for the pla...
 Name: 199999, dtype: object,
 91)

# BASELINE

In [0]:
!pip install transformers

In [36]:
import numpy as np
from tqdm import tqdm_notebook

import torch
# from torch.utils.data import DataLoader
from torchtext  import data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# from torchtext.vocab import Vectors

from transformers import BertTokenizer, BertModel, GPT2Model, GPT2Tokenizer

import nltk
from nltk import tokenize
nltk.download('punkt')

SEED = 23
np.random.seed(SEED)
torch.manual_seed(SEED)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [37]:
pretrained_weights = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(pretrained_weights)
model = BertModel.from_pretrained(pretrained_weights)


embeddings_pretrained = model.get_input_embeddings()
embeddings_pretrained

def tokenize(text, tokenizer=tokenizer):
    return tokenizer.encode(text, max_length=512)

In [0]:
# count = 0
# neg_count = 0
# for el in X_train['text']:
#     if len(tokenize(el)) == 0:
#         count += 1
#         print(el)
#         break
#     else:
#         neg_count += 1
# count, neg_count

In [38]:
MAX_VOCAB_SIZE = 50000
classes={'fake': 0, 'real': 1}


SCORE = data.Field(dtype=torch.float, use_vocab=True)


TEXT = data.Field(sequential=True, 
                  include_lengths=False,
                  batch_first=True, 
                  tokenize=tokenize, 
                  pad_first=True, 
                  lower=False) 


train = data.TabularDataset('/content/test.csv', format='csv', 
                            fields=[('text', TEXT), ('parent_text', TEXT), ('label',SCORE)], 
                                skip_header=True)

test = data.TabularDataset('/content/test.csv',  format='csv', fields=[('text', TEXT), ('parent_text', TEXT), ('label',SCORE)], 
                                skip_header=True)

TEXT.build_vocab(train,  max_size=MAX_VOCAB_SIZE, min_freq=2)
SCORE.build_vocab(train)
vocab = TEXT.vocab
print('Vocab size:', len(TEXT.vocab.itos))

train, valid = train.split(0.8)

Vocab size: 26175


In [39]:
len(train), len(valid)

(159805, 39951)

In [0]:
# def check_nan(dataset, column, train=True):
#     for ind, el in enumerate(dataset):
#         if column not in el.__dict__.keys():
#             print(train, el.__dict__, ind)

# for i, d in enumerate([train, test]):
#     for el in ['text', 'parent_text', 'label']:
#         flag = True if i==0 else False
#         check_nan(d, el, flag)

In [0]:
EMBEDDINGS_DIM = embeddings_pretrained.embedding_dim
VOCAB_SIZE = embeddings_pretrained.num_embeddings
EMB_PRETRAINED = True

In [0]:
class MyModel(nn.Module):
    
    def __init__(self, vocab_size, embed_size, hidden_size, 
                 emb_pretrained, embeddings):
        super(MyModel, self).__init__()
        self.emb_pretrained = emb_pretrained
        self.embedding =  embeddings if self.emb_pretrained else nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.LSTM(input_size=embed_size,
                           hidden_size=hidden_size,
                           bidirectional=True,
                           batch_first=True,
                          )
        
        self.fc = nn.Linear(hidden_size * 2 *2, 1)
    def forward(self, x):
        
        x = self.embedding(x)
           
        _, (hidden, cell) = self.rnn(x)
        
        hidden = hidden.transpose(0,1)
        cell = cell.transpose(0,1)
        hidden = hidden.contiguous().view(hidden.size(0),-1)
        cell = cell.contiguous().view(cell.size(0),-1)
        x = torch.cat([hidden, cell], dim=1).squeeze(1)
        x = self.fc(x)
        return x

In [45]:
BATCH_SIZE = 64
num_epochs = 10

model = MyModel(VOCAB_SIZE,
                embed_size=EMBEDDINGS_DIM,
                hidden_size=256,
                emb_pretrained = EMB_PRETRAINED,
                embeddings = embeddings_pretrained
               )
model.to(device)
# wandb.watch(model)


train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, valid, test),
    batch_sizes=(BATCH_SIZE, BATCH_SIZE, BATCH_SIZE),
    shuffle=True,
    device=device,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
)

optimizer = optim.Adam(model.parameters(), weight_decay=1e-5)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2, verbose=True, cooldown=5)
criterion = torch.nn.MSELoss()
criterion.to(device)

BCEWithLogitsLoss()

In [0]:
def _train_epoch(model, iterator, optimizer, criterion, curr_epoch):
    model.train()

    running_loss = 0
    losses = []
    # train_acc = []

    n_batches = len(iterator)
    
    iterator = tqdm_notebook(iterator, total=n_batches, desc='epoch %d' % (curr_epoch), leave=True)
    
    for i, batch in enumerate(iterator):
        x = batch.text
        y = batch.label.squeeze(0)
        optimizer.zero_grad()
        preds = model(x).squeeze(1)
        loss = criterion(preds, y)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
        optimizer.step()
        
        curr_loss = loss.data.detach().item()
        losses.append(loss.item())

        # acc = accuracy_score(preds, y)
        # train_acc.append(acc)

        iterator.set_postfix(loss='%.5f' % curr_loss)

        # wandb.log({
        # "Train Accuracy": np.mean(train_acc),
        # "Train Loss": curr_loss})

    return curr_loss, losses

def _test_epoch(model, iterator, criterion):
    model.eval()
    
    epoch_loss = 0
    losses = []
    # test_acc = []

    n_batches = len(iterator)

    with torch.no_grad():
        for batch in iterator:
            x = batch.text
            y = batch.label.squeeze(0)
            preds = model(x).squeeze(1)
            # test_acc.append(accuracy_score(preds, y))

            loss = criterion(preds, y)
            losses.append(loss.item())
            epoch_loss += loss.data.item()

    # wandb.log({
    #     "Valid Accuracy": np.mean(test_acc),
    #     "Valid Loss": epoch_loss/n_batches})
    
    return epoch_loss / n_batches, losses

def nn_train(model, train_iterator, valid_iterator, criterion, optimizer, n_epochs=20, early_stopping=0):

    prev_loss = 10500
    es_epochs = 0
    best_epoch = None
    history = pd.DataFrame()

    train_losses = []
    valid_losses = []

    # train_accs = []
    # valid_accs = []

    for epoch in range(n_epochs):
        train_loss, epoch_tl = _train_epoch(model, train_iterator, optimizer, criterion, epoch)
        valid_loss,  epoch_vl = _test_epoch(model, valid_iterator, criterion)

        train_losses.extend(epoch_tl)
        valid_losses.extend(epoch_vl)
        # train_accs.extend(train_acc)
        # valid_accs.extend(valid_acc)

        print('validation loss %.5f' % valid_loss)

        record = {'epoch': epoch, 'train_loss': train_loss, 'valid_loss': valid_loss}
        history = history.append(record, ignore_index=True)

        if early_stopping > 0:
            if valid_loss > prev_loss:
                es_epochs += 1
            else:
                es_epochs = 0

            if es_epochs >= early_stopping:
                best_epoch = history[history.valid_loss == history.valid_loss.min()].iloc[0]
                print('Early stopping! best epoch: %d val %.5f' % (best_epoch['epoch'], best_epoch['valid_loss']))
                break

            prev_loss = min(prev_loss, valid_loss)
    return history

In [63]:
history = nn_train(model, train_iterator, valid_iterator,
          criterion, optimizer, n_epochs=100, early_stopping=2)


validation loss -4669976.83895


KeyboardInterrupt: ignored

In [56]:
a = next(iter(train_iterator))
model.forward(a.text).squeeze(1).size()

torch.Size([64])

In [59]:
a.label.squeeze(0).size()

torch.Size([64])

In [57]:
a.label

tensor([[ 272.,   22.,   42.,   58.,    3.,   56.,   67.,   56., 1008.,  283.,
           12.,   22.,   40.,    2., 1912.,   76.,  250.,    8.,   11.,    2.,
           84.,    2.,   49.,    4.,    5.,    4.,    4.,  179.,   16.,   36.,
            3.,  256.,   48.,  107.,  256.,    5.,    2.,   38.,   54.,    2.,
            4.,    4.,   35.,  214.,   75.,   12.,   60.,    2.,   28.,   30.,
           64.,   35.,  222.,   94.,    9.,    4.,  237.,   35.,    7.,    2.,
           65.,  248.,   20.,   44.]], device='cuda:0')